In [1]:
%load_ext autoreload
%autoreload 2

In [83]:
import os.path
import keras
import numpy as np
import time
from matplotlib import pyplot as plt
from fnn_helper import PlotLosses
from MNIST_helper import plot_numbers, visualize_input
from keras.datasets import mnist
from keras.models import Sequential, Model
from keras.layers import Flatten, Dense, Activation
from keras.optimizers import Adam, SGD
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.layers.advanced_activations import LeakyReLU
from fnn_helper import PlotLosses
from keras_contrib.callbacks import CyclicLR
from matplotlib import pyplot as plt
from mlp_helper import plot_confusion_matrix
import draw_nn
import keras.backend as K
if not os.path.exists('mlp_helper.py'):
    !wget https://github.com/lab-ml-itba/MLP-2019/raw/master/mlp_helper.py
    !wget https://github.com/lab-ml-itba/MLP-2019/raw/master/fnn_helper.py
    !wget https://github.com/lab-ml-itba/MLP-2019/raw/master/MNIST_helper.py
    !wget https://github.com/lab-ml-itba/MLP-2019/raw/master/draw_nn.py

# Split data

In [18]:
(x, y), (x_test, y_test) = mnist.load_data()

# 20% del dataset para validación
validation = 0.20
N_validation_split = int(x.shape[0]*(1-validation))
# Training Set
x_train = x[:N_validation_split]
y_train = y[:N_validation_split]

# Cross Validation Set
x_val = x[N_validation_split:]
y_val = y[N_validation_split:]
print('Training set data:', x_train.shape)
print('Training labels:', y_train.shape)
print()
print('CV set data:', x_val.shape)
print('CV labels:', y_val.shape)
print()
print('Testing set data:',x_test.shape)
print('Testing labels:', y_test.shape)

Training set data: (48000, 28, 28)
Training labels: (48000,)

CV set data: (12000, 28, 28)
CV labels: (12000,)

Testing set data: (10000, 28, 28)
Testing labels: (10000,)


# One-hot encoding

In [19]:
y_train_categorical = keras.utils.to_categorical(y_train)
y_val_categorical = keras.utils.to_categorical(y_val)
y_test_categorical = keras.utils.to_categorical(y_test)
print(y_train_categorical.shape)
print(y_train_categorical[:5])

(48000, 10)
[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


# Modelo

In [84]:
def get_model(units_in_hidden_2=2):
    output_size = 10
    model_plane = Sequential()
    model_plane.add(Flatten(input_shape=x_train.shape[1:]))
    model_plane.add(Dense(200, activation='relu',  name='middle1', kernel_initializer='normal'))
    # model_plane.add(LeakyReLU())
    model_plane.add(Dense(units_in_hidden_2, activation='relu', name='middle2', kernel_initializer='normal'))
    # model_plane.add(LeakyReLU())
    model_plane.add(Dense(output_size, activation='softmax',  kernel_initializer='normal', name='salida'))
    return model_plane

In [85]:
model_plane = get_model(2)
model_plane.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_20 (Flatten)         (None, 784)               0         
_________________________________________________________________
middle1 (Dense)              (None, 200)               157000    
_________________________________________________________________
middle2 (Dense)              (None, 2)                 402       
_________________________________________________________________
salida (Dense)               (None, 10)                30        
Total params: 157,432
Trainable params: 157,432
Non-trainable params: 0
_________________________________________________________________


In [104]:
K.clear_session()
batch_size=12
lr = 1e-4
optim = Adam(lr=lr)
units_in_hidden_2 = 2
model_plane = get_model(units_in_hidden_2)

In [105]:
optim_name = str(type(optim)).replace("'>", '').split('.')[-1]
log_dir_name = './tb-params-test/'+optim_name+'_'+str(batch_size)+'_'+str(lr)+'_'+str(units_in_hidden_2)+'_'+str(int(time.time()))
print(log_dir_name)

./tb-params-test/Adam_64_0.0001_2_1559849832


In [106]:
# checkpointer = ModelCheckpoint(filepath='param-tests.mnist.hdf5', verbose=0, save_best_only=True, mode='max', monitor='val_acc')
tbCB = TensorBoard(log_dir=log_dir_name, histogram_freq=1, batch_size=batch_size, write_graph=False, 
                   write_grads=True, write_images=False, embeddings_freq=0, 
                   embeddings_layer_names=None, embeddings_metadata=None, embeddings_data=None, 
                   update_freq='epoch')

# clr = CyclicLR(1e-6, 1e-4, 8*int(len(x_train)/batch_size))
model_plane.compile(loss = 'categorical_crossentropy', optimizer=optim, metrics=['accuracy'])

In [ ]:
model_plane.fit(x_train, 
          y_train_categorical ,
          epochs=100, batch_size=batch_size, 
          verbose=0, 
          validation_data=(x_test, y_test_categorical), 
          callbacks=[tbCB],
         )

In [93]:
model_plane.evaluate(x_train, y_train_categorical)

48000/48000 [==============================] - 1s 17us/step


[0.02440831386235853, 0.9976458333333333]

In [94]:
model_plane.evaluate(x_test, y_test_categorical)

10000/10000 [==============================] - 0s 19us/step


[0.49745430337451396, 0.9454]

In [124]:
# model_plane.load_weights('two-layer-2d-leaky-relu.mnist.hdf5')
model_plane.load_weights('two-layer-2d-relu.mnist.hdf5')
model_plane.evaluate(x_test, y_test_categorical)

10000/10000 [==============================] - 0s 39us/step


[0.44512823535501955, 0.9496]